In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

### 데이터셋 불러오기

In [2]:
training_data = datasets.FashionMNIST(
    root="D:/mycode/PyTorch_Tutorials//data",  # 데이터 저장 경로
    train=True,  # 학습용인지 테스트용인지
    download=True, # root에 데이터가 없으면 새로 다운
    transform=ToTensor() # 데이터를 텐서 형태로 변환
)

test_data = datasets.FashionMNIST(
    root="D:/mycode/PyTorch_Tutorials//data",
    train=False,
    download=True,
    transform=ToTensor()
)

### 파일에서 사용자 정의 데이터셋 만들기

In [3]:
# 파이토치 Dataset클래스에서는 __init__, __len__, __getitem__ 은 무조건 필요하다.
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])  # annotations_file: 정답 저장 파일명
        self.img_dir = img_dir   # 이미지 파일 저장 폴더경로
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self): # 데이터셋의 샘플 갯수 반환
        return len(self.img_labels)

    def __getitem__(self, idx): # 주어진 idx(인덱스)에 해당하는 데이터셋을 불어옴
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)  # read_image: 이미지를 텐서로 변환
        label = self.img_labels.iloc[idx, 1] # label을 가져옴
        if self.transform:    # transform=True, target_transform=True 인 경우 텐서이미지와 라벨을 dict형으로 반환
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### DataLoader로 학습용 데이터 준비하기

In [4]:
# Dataset: 데이터셋의 피쳐와 라벨을 지정
# DataLoader: Dataset에 미니배치 및 셔플 설정
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
